In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

2023-01-14 11:59:03.770746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-14 11:59:04.281700: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-14 11:59:04.281719: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-14 11:59:04.340361: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-14 11:59:05.408575: W tensorflow/stream_executor/platform/de

In [ ]:
pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 545.6 kB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
     ━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━ 20.1/34.4 MB 402.0 kB/s eta 0:00:36

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from tensorflow.keras.layers import TextVectorization
import string
import re

In [ ]:
import matplotlib.pyplot as plt
import numpy

In [ ]:
data = pd.read_csv("new_data.csv",)
data.dropna()
data.head()

In [ ]:
val_dataframe = data.sample(frac=0.2, random_state=1337)
train_dataframe = data.drop(val_dataframe.index)

def dataframe_to_dataset(data):
    data = data.copy()
    labels = data.pop("Sentiment positive")
    ds = tf.data.Dataset.from_tensor_slices((data["String"], labels))
    ds = ds.shuffle(buffer_size=len(data))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [ ]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    lowercase = tf.strings.strip(lowercase) 
    lowercase = tf.strings.regex_replace(lowercase, "<br />", " ")
    lowercase = tf.strings.regex_replace(lowercase, "#", " ")
    lowercase = tf.strings.regex_replace(lowercase, 
                                         r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', " ")
    lowercase = tf.strings.regex_replace(lowercase,
    r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", " ")
    stripped_html = tf.strings.regex_replace(lowercase, "<[^>]+>", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )

max_features = 2000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_ds = train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

In [ ]:
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features+1, embedding_dim)(x)
x = layers.Dropout(0.5)(x)

# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# x = layers.Flatten()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)
model = tf.keras.Model(text_input, predictions)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
checkpoint_filepath = 'Basic_model/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
tb_callback = tf.keras.callbacks.TensorBoard('Basic_LSTM_model/logs', update_freq=1)
my_callbacks = [
    model_checkpoint_callback,
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='Basic_model/model/model.{epoch:02d}-{val_loss:.2f}.tf'),
#     tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tb_callback,
]

In [ ]:
epochs = 1

# Fit the model using the train and test datasets.
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs,callbacks=my_callbacks)

In [ ]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
import io

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0:
        continue  # skip 0, it's padding.
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()